In [1]:
from thespian.actors import *

class Hello(Actor):
    def receiveMessage(self, msg, sender):
        self.send(sender, 'Hello world!')

asys = ActorSystem()

hello = asys.createActor(Hello)

In [2]:
ActorSystem().ask(hello, 'hi', 1)

'Hello world!'

In [3]:
ActorSystem().tell(hello, ActorExitRequest())

In [4]:
hello

In [5]:
import sys

class Greeting(object):

    def __init__(self, msg):
        self.message = msg

    def __str__(self):
        return self.message

class World(Actor):

    def receiveMessage(self, msg, sender):
        if isinstance(msg, Greeting):
            msg.message = msg.message + ", World"
            nextTo = msg.sendTo.pop(0)
            self.send(nextTo, msg)


class Punctuate(Actor):

    def receiveMessage(self, msg, sender):
        if isinstance(msg, Greeting):
            msg.message = msg.message + "!"
            nextTo = msg.sendTo.pop(0)
            self.send(nextTo, msg)


class Hello2(Actor):
    def receiveMessage(self, msg, sender):
        if msg == 'hi':
            greeting = Greeting('Hello')
            world = self.createActor(World)
            punct = self.createActor(Punctuate)
            greeting.sendTo = [punct, sender]
            self.send(world, greeting)

hello = ActorSystem().createActor(Hello2)

hello

In [6]:
ActorSystem().ask(hello, 'hi', 0.2)

'Hello world!'

In [7]:
ActorSystem().tell(hello, ActorExitRequest())

In [8]:
ActorSystem().ask(hello, 'hi', 0.2)

In [9]:
hello = ActorSystem().createActor(Hello2)

In [10]:
ActorSystem().ask(hello, 'hi', 0.2)

In [11]:
ActorSystem().ask(hello, 'hi', 0.2)

In [12]:
ActorSystem().ask(hello, 'hi', 0.2).message

'Hello, World!'

In [13]:
ActorSystem().ask(hello, 'hi', 0.2).message

'Hello, World!'

In [14]:
import time
def timeit(func):
    def _timer(*args, **kw):
        start = time.time()
        ret = func(*args, **kw)
        end = time.time()
        print('%s(%s, %s) -- %2.2f sec' % (func.__name__, args, kw, end-start))
        return ret
    return _timer

from thespian.troupe import troupe
from datetime import timedelta

max_wait = timedelta(seconds=60)


def fib(n):
    if n <= 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)


class Fibonacci(ActorTypeDispatcher):
    def receiveMsg_int(self, intval, sender):
        self.send(sender, fib(intval))


@troupe(10)
class FibonacciT(ActorTypeDispatcher):
    def receiveMsg_int(self, intval, sender):
        self.send(sender, fib(intval))

def serial_test(asys, fibber, inputs):
    for N in inputs:
        print(asys.ask(fibber, N, max_wait))
    asys.tell(fibber, ActorExitRequest())

def parallel_test(asys, fibber, inputs):
    for N in inputs:
        asys.tell(fibber, N)
    for R in range(len(inputs)):
        print(asys.listen(max_wait))
    asys.tell(fibber, ActorExitRequest())


@timeit
def t1(asys, inputs):
    serial_test(asys, asys.createActor(Fibonacci), inputs)

@timeit
def t2(asys, inputs):
    parallel_test(asys, asys.createActor(Fibonacci), inputs)

@timeit
def t3(asys, inputs):
    serial_test(asys, asys.createActor(FibonacciT), inputs)


@timeit
def t4(asys, inputs):
    parallel_test(asys, asys.createActor(FibonacciT), inputs)

asys = ActorSystem('multiprocQueueBase')
inputs = (10, 20, 35, 15, 35, 34, 33, 36)

In [15]:
t1(asys, inputs)

55
6765
9227465
610
9227465
5702887
3524578
14930352
t1((<thespian.actors.ActorSystem object at 0x10bfd3070>, (10, 20, 35, 15, 35, 34, 33, 36)), {}) -- 11.37 sec


In [16]:
t2(asys, inputs)

55
6765
9227465
610
9227465
5702887
3524578
14930352
t2((<thespian.actors.ActorSystem object at 0x10bfd3070>, (10, 20, 35, 15, 35, 34, 33, 36)), {}) -- 11.27 sec


In [17]:
t3(asys, inputs)

55
6765
9227465
610
9227465
5702887
3524578
14930352
t3((<thespian.actors.ActorSystem object at 0x10bfd3070>, (10, 20, 35, 15, 35, 34, 33, 36)), {}) -- 10.60 sec


In [18]:
t4(asys, inputs)

55
6765
610
3524578
5702887
9227465
9227465
14930352
t4((<thespian.actors.ActorSystem object at 0x10bfd3070>, (10, 20, 35, 15, 35, 34, 33, 36)), {}) -- 3.60 sec


In [19]:
asys.shutdown()